In [ ]:
# import inspect,sys,pprint
# import apache_beam as beam

# print pprint.pprint(inspect.getmembers(sys.modules['apache_beam.io']))
# print pprint.pprint(beam.options.pipeline_options.PipelineOptions().get_all_options())

In [10]:
# %load beam.py
import tempfile
import json

import apache_beam as beam
import tensorflow as tf

import tensorflow_transform.beam.impl as beam_impl
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.beam.tft_beam_io import beam_metadata_io


def preprocess():
    pipeline_options = beam.pipeline.PipelineOptions(flags=[], **{
        'job_name': 'test',
        'project': 'srfrnk-test',
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
    })

    with beam.Pipeline('DirectRunner', options=pipeline_options) as pipeline:
        with beam_impl.Context(temp_dir=tempfile.mkdtemp()):
            text_lines = pipeline | 'read' >> beam.io.ReadFromText(
                './weather.json')

            parsed_lines = text_lines | 'parse' >> beam.Map(
                lambda (text_line): json.loads(text_line))

            raw_data = parsed_lines | 'map' >> beam.Map(lambda (line): {
                'stn': int(line['stn']),
                'year': int(line['year']),
                'mo': int(line['mo']),
                'da': int(line['da']),
                'temp': line['temp'],
            })

            def preprocessing_fn(inputs):
                stn = inputs['stn']
                year = inputs['year']
                mo = inputs['mo']
                da = inputs['da']
                temp = inputs['temp']

                # stn = stn - tft.mean(stn)
            #   y_normalized = tft.scale_to_0_1(y)
            #   s_integerized = tft.string_to_int(s)
            #   x_centered_times_y_normalized = (x_centered * y_normalized)
                return {
                    'stn': stn,
                    'year': year,
                    'mo': mo,
                    'da': da,
                    'temp': temp
                }

            raw_md = dataset_metadata.DatasetMetadata(dataset_schema.Schema({
                'stn': dataset_schema.ColumnSchema(tf.int64, [], dataset_schema.FixedColumnRepresentation()),
                'year': dataset_schema.ColumnSchema(tf.int64, [], dataset_schema.FixedColumnRepresentation()),
                'mo': dataset_schema.ColumnSchema(tf.int64, [], dataset_schema.FixedColumnRepresentation()),
                'da': dataset_schema.ColumnSchema(tf.int64, [], dataset_schema.FixedColumnRepresentation()),
                'temp': dataset_schema.ColumnSchema(tf.float32, [], dataset_schema.FixedColumnRepresentation())
            }))

            # (trans_data, trans_md), trans_fn = (
            (trans_data, trans_md), _ = (
                (raw_data, raw_md) | beam_impl.AnalyzeAndTransformDataset(preprocessing_fn))

            trans_data | 'write data' >> beam.io.WriteToTFRecord(
                './data', coder=example_proto_coder.ExampleProtoCoder(trans_md.schema))
            trans_md | 'write metadata' >> beam_metadata_io.WriteMetadata(
                './metadata', pipeline=pipeline)


preprocess()


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /tmp/tmpk_DspS/tftransform_tmp/0d544679bc7249a7b41cf6f59f2928a2/saved_model.pb


INFO:tensorflow:SavedModel written to: /tmp/tmpk_DspS/tftransform_tmp/0d544679bc7249a7b41cf6f59f2928a2/saved_model.pb


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
input_form = """
<table>
<td style="border-style: none;">
<div style="border: solid 2px #666; width: 143px; height: 144px;">
<canvas width="140" height="140"></canvas>
</div></td>
<td style="border-style: none;">
<button onclick="clear_value()">Clear</button>
</td>
</table>
"""

javascript = '''
<script type="text/Javascript">
    var pixels = [];
    for (var i = 0; i < 28*28; i++) pixels[i] = 0;
    var click = 0;

    var canvas = document.querySelector("canvas");
    canvas.addEventListener("mousemove", function(e){
        if (e.buttons == 1) {
            click = 1;
            canvas.getContext("2d").fillStyle = "rgb(0,0,0)";
            canvas.getContext("2d").fillRect(e.offsetX, e.offsetY, 8, 8);
            x = Math.floor(e.offsetY * 0.2);
            y = Math.floor(e.offsetX * 0.2) + 1;
            for (var dy = 0; dy < 2; dy++){
                for (var dx = 0; dx < 2; dx++){
                    if ((x + dx < 28) && (y + dy < 28)){
                        pixels[(y+dy)+(x+dx)*28] = 1;
                    }
                }
            }
        } else {
            if (click == 1) set_value();
            click = 0;
        }
    });
    
    function set_value(){
        var result = ""
        for (var i = 0; i < 28*28; i++) result += pixels[i] + ","
        var kernel = IPython.notebook.kernel;
        kernel.execute("image = [" + result + "]");
        kernel.execute("f = open('request.json', 'w')");
        kernel.execute("f.write('{\\"inputs\\":%s}' % image)");
        kernel.execute("f.close()");
    }
    
    function clear_value(){
        canvas.getContext("2d").fillStyle = "rgb(255,255,255)";
        canvas.getContext("2d").fillRect(0, 0, 140, 140);
        for (var i = 0; i < 28*28; i++) pixels[i] = 0;
    }
</script>
'''

from IPython.display import HTML
HTML(input_form + javascript)